In [ ]:
import mne
import numpy as np

import sys
import copy
import pickle

sys.path.append('..')

from _parameters import *
sys.path.remove('..')

from IPython.display import clear_output

In [ ]:
def open_tfr(s, moment, modal):
    
    # Dict per cond
    tfr = {c: None for c in cond[modal]}

    # Read tfr per cond
    for c in cond[modal]:

        tfr_fname = dirs['tfr'] + '/tfr_' + moment + '_' + c.replace('/', '_') + '_s' + str(s) + '.h5'    

        # Add to dict
        tfr[c] = mne.time_frequency.read_tfrs(fname=tfr_fname)[0]

    return tfr

In [ ]:
def calc_cvsi(tfr, trials_L, trials_R, chan_L, chan_R):

    tfr_cpy = copy.deepcopy(tfr)

    # Left channels
    contra_L = tfr_cpy[trials_R].pick(chan_L).data.mean(0) # contra
    ipsi_L = tfr_cpy[trials_L].pick(chan_L).data.mean(0) # ipsi

    cvsi_L = ((contra_L - ipsi_L) / (contra_L + ipsi_L)) * 100

    tfr_cpy = copy.deepcopy(tfr)
    
    # Right channels
    contra_R = tfr_cpy[trials_L].pick(chan_R).data.mean(0) # contra
    ipsi_R = tfr_cpy[trials_R].pick(chan_R).data.mean(0) # ipsi

    cvsi_R = ((contra_R - ipsi_R) / (contra_R + ipsi_R)) * 100

    # Average
    cvsi = np.mean(np.asarray([cvsi_L, cvsi_R]), 0)

    return cvsi

In [ ]:
def get_cvsi(subs, moment):

    # Empty list for each cond
    cvsi_cond = ['vis/same', 'vis/rvrs', 'mot/same', 'mot/rvrs']
    cvsi_dat = {c:[] for c in cvsi_cond}

    # Loop over subs
    for s in subs:

        # Get tfr data
        tfr_vis = open_tfr(s, moment, 'visual')
        tfr_mot = open_tfr(s, moment, 'motor')

        # calc cvsi per cond
        cvsi_dat['vis/same'].append(
            calc_cvsi(tfr_vis, 'same/itemL', 'same/itemR', 'PO7', 'PO8')
        )
        cvsi_dat['vis/rvrs'].append(
            calc_cvsi(tfr_vis, 'rvrs/itemL', 'rvrs/itemR', 'PO7', 'PO8')
        )
        cvsi_dat['mot/same'].append(
            calc_cvsi(tfr_mot, 'same/respL', 'same/respR', 'C3', 'C4')
        )
        cvsi_dat['mot/rvrs'].append(
            calc_cvsi(tfr_mot, 'rvrs/respL', 'rvrs/respR', 'C3', 'C4')
        )

        clear_output(wait = False)

    # Avg over subs
    cvsi_avg = {c:None for c in cvsi_cond}

    for c in cvsi_dat.keys():
        cvsi_avg[c] = np.mean(np.asarray(cvsi_dat[c]),0)

    return cvsi_dat, cvsi_avg

In [ ]:
# Run all

# Get cvsi for subs
cvsi_dat, cvsi_avg = get_cvsi(subjects, moment = 'enc1')

# Save as pickle
cvsi_dat_f = open(dirs['cvsi'] + '/cvsi_enc1.pkl', 'wb')

pickle.dump([cvsi_dat, cvsi_avg], cvsi_dat_f)
cvsi_dat_f.close()